In [1]:
import errno
import os
import math
import signal
import functools
import random
from sympy import sympify
from sympy.functions import Abs
from sympy.core.rules import Transform
import json
import time
from collections import Counter

ops = ["sqrt", "cbrt", "sin", "sinh", "asin", "cos", "cosh", "acos", "tan", "cot", "tanh", "sec", "csc", "atan", "log", "exp"]

class TimeoutError(Exception):
    pass

def timeout(seconds=15, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result
        return wrapper
    return decorator

def round_all_floats(f, num_digits=2):
    return f.xreplace(Transform(lambda x: x.round(num_digits), lambda x: isinstance(x, sp.Float)))

def get_all_constants(expr):
    return [t for t in expr.atoms() if t not in expr.free_symbols]

# Return coeffs of a polynomial from lowest power to highest
def get_polynomial_coeffs(f, max_power=6):
    result = [0.0]*(max_power+1)
    coeff_pairs = get_coefficients_and_exponents(f)
    for p in coeff_pairs:
        coeff, exp = float(p[0]), float(p[1])
        idx = int(round(exp))
        if idx < 0 or idx > max_power:
            continue
        result[idx] = coeff
    return result

# Only keep the polynomial part
@timeout(5)
def filter_non_polynomial(f):
    variables = list(f.free_symbols)
    assert len(variables)<=1, "Expression having multiple variable " + str(f)
    if len(variables) == 0:
        return sympify(0)
    t = variables[0]
    terms_kept = []
    terms = f.as_ordered_terms()
    for term in terms:
        coeff, exp = term.as_coeff_exponent(t)
        if coeff.is_constant():
            terms_kept.append(term)
    if len(terms_kept) == 0:
        return sympify(0)
    return sum(terms_kept)

def shrink_a_number(n):
    if n.is_Integer and Abs(n) > 20:
        return n // (10**math.floor(math.log10(abs(n))))
    elif n.is_Float and Abs(n) > 20:
        return n / (10**math.floor(math.log10(abs(n))))
    elif n.is_Rational and Abs(n) > 20:
        return n // (10**math.floor(math.log10(abs(n))))
    return n

# Randomize a number
# if shrink==True, shrink the number to be within 10
def randomize_number(n):
    if n.is_Integer:
        for i in range(10):
            if i % 2:
                m = n + random.randint(-2,2)
            else:
                m = n * random.uniform(0.5, 2)
            if (n > 0 and m > 0) or (n < 0 and m < 0):
                return m
        return n
    elif n.is_Float:
        m = n * random.choice([sympify('1/2'), 2, sympify('1/3'), 3, sympify('2/3'), sympify('3/2'), sympify('1/4'), 4, sympify('3/4'), sympify('4/3'), sympify('1/5'), 5, sympify('2/5'), sympify('5/2'), sympify('3/5'), sympify('5/3'), sympify('4/5'), sympify('5/4'), sympify('1/6'), 6, sympify('5/6'), sympify('6/5')])
        return m
    return n

def random_remove_items(expr, prob_delete=0.1):
    for i in range(10):
        constants = [t for t in expr.atoms() if t not in expr.free_symbols]
        mapping = {}
        for constant in constants:
            if random.random() < prob_delete:
                mapping[constant] = sympify('0')
        expr_removed = expr.subs(mapping)
        if not expr_removed.is_constant():
            return expr_removed
    return expr

def shrink_numbers(expr):
    constants = [t for t in expr.atoms() if t not in expr.free_symbols]
    mapping = {}
    for constant in constants:
        mapping[constant] = shrink_a_number(constant)
    return expr.subs(mapping)

def randomize(expr, prob_change=0.5, shrink=False, remove_items=False):
    return_str = False
    if type(expr) == type(""):
        return_str = True
        expr = sp.sympify(expr)
    if shrink:
        expr = shrink_numbers(expr)
    constants = [t for t in expr.atoms() if t not in expr.free_symbols]
    mapping = {}
    for constant in constants:
        if random.random() < prob_change:
            mapping[constant] = randomize_number(constant)
    expr_subed = expr.subs(mapping)
    if remove_items:
        expr_subed = random_remove_items(expr_subed)
    if return_str:
        expr_subed = str(expr_subed)
        return expr_subed
    else:
        return expr_subed

def normalize_expr(expr, range_start, range_end, points):
    center = (range_end + range_start) / 2
    step = 2 * (range_end - center) / (points)
    func = eval("lambda x: " + expr)
    total = 0
    real_points = 0
    min_y = 2147483647
    max_y = -2147483647
    try:
        total += func(center)
        min_y = min(min_y, total)
        max_y = max(max_y, total)
    except:
        pass
    x = step
    while x <= range_end:
        try:
            res = func(x)
            total += res
            min_y = min(min_y, res)
            max_y = max(max_y, res)
            real_points += 1
        except:
            pass
        x = round(x + step, 5)
    x = -step
    while x >= range_start:
        try:
            res = func(x)
            total += res
            min_y = min(min_y, res)
            max_y = max(max_y, res)
            real_points += 1
        except:
            pass
        x = round(x - step, 5)
    avg = total / real_points
    if avg != 0:
        expr += "-" + str(avg)
    if avg > 0:
        min_y -= avg
    else:
        max_y += avg
    abs_range = random.uniform(0.5, 1.5)
    coeff = abs_range / max(abs(min_y), abs(max_y))
    expr = "(" + expr + ")" + "*" + str(coeff)
    expr = str(sp.sympify(expr))
    return expr

def truncate(expr, sig_digits):
    expr = sp.sympify(expr)
    res = expr
    for a in sp.preorder_traversal(expr):
        if isinstance(a, sp.Float):
            round_digits = sig_digits - len(str(a).split(".")[0])
            res = res.subs(a, round(a, max(0, round_digits)))
    return str(res)

def generate_quads(filepath):
    eq = str(random.uniform(-10, 10)) + "*x**2" + "+" + str(random.uniform(-10, 10)) + "*x" + "+" + str(random.uniform(10, -10))
    processed_eq = truncate_str(normalize_expr(eq, -4, 4, 40), 4)
    with open(filepath, "a") as outfile:
        json.dump({"question": "[BOS]", "answer": processed_eq}, outfile)
        outfile.write("\n")
    return processed_eq

def is_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

In [9]:
import sympy as sp
from sympy import symbols
import json
from datasets import load_dataset, Dataset, load_from_disk

coeff_letters = "abcdefghijklmnopqrstuvwxyz"
@timeout(2)
def get_coeffs(expr):
    expr = sympify(expr)
    # print("expr", expr)
    expanded_expr = sp.expand(expr)
    # Split the expression string into individual operations

    # Use as_coefficients_dict() to parse out coefficients and terms
    term_dict = expanded_expr.as_coefficients_dict()
    coeffs = []
    if len(term_dict) == 1:
        if isinstance(expr, (sp.Integer, sp.Float)):
            return [str(expr).rstrip("0")]
        if isinstance(expr, (sp.Pow)):
            return []
        else:
            for arg in expr.args:
                coeffs += get_coeffs(arg)
            return coeffs
    # Print the coefficients and corresponding terms
    for coefficient, term in term_dict.items():
        coefficient = str(coefficient)
        term = str(term)
        if is_float(coefficient):
            coeffs += get_coeffs(term)
            if coefficient != "1":
                coeffs.append(coefficient.rstrip("0"))
        elif is_float(term):
            coeffs += get_coeffs(coefficient)
            if term != "1":
                coeffs.append(term.rstrip("0"))
    return coeffs
def get_template(expr, coeffs):
    mapping = []
    coeff_idx = 0
    for coeff in coeffs:
        if "." in coeff:
            coeff = float(coeff)
        else:
            coeff = int(coeff)
        mapping.append((coeff, coeff_letters[coeff_idx]))
        coeff_idx += 1
    # print(mapping)
    return expr.subs(mapping)
        
# get_template("(-6*t**2 - 3)**(1/3)")
fin = open("/home/mcwave/code/automath/calculus/datasets/function_evaluation.json")
lines = fin.readlines()
fin.close()

templates = {}

sexprs = []
for i in range(100): #len(lines)):
    data = json.loads(lines[i])
    try:
        s = data["f_t"][1]
        sexprs.append(s)
        expr = sp.sympify(s)
        template = get_template(expr, get_coeffs(expr))
        if template not in templates:
            templates[template] = 0
        templates[template] += 1
        print(expr)
        print(template)
    except:
        print("Cannot process", line)
    if i % 1000 == 0:
        print(i, "cases processed")

-tan(7*t - 7)
-a*tan(b*t - b)
0 cases processed
tan(6*t - 1)
tan(a + b*t)
-sin(2*t + 7)/(-4*t - 4)**0.333333333333333
a*sin(b + c*t)/(-4*t - 4)**0.333333333333333
-(6*t - 5)**0.333333333333333 - sin(6*t**5 - 9)
a*(c*t - 5)**0.333333333333333 + a*sin(b + c*t**5)
-tan(5*t - 5)
-a*tan(b*t - b)
(-6*t**2 - 3)**(1/3)
(-6*t**2 - 3)**(1/3)
sin(16*t/3 + 6.66666666666667)
sin(a + 16*t/3)
(-6*t - 7)**(-4)
(-6*t - 7)**(-4)
-cos(5*t - 6) - tanh(2*t + 7)
c*cos(a + b*t) + c*tanh(d + e*t)
-tan(4*t - 5)
a*tan(b + c*t)
cos(6*t + 0.5)
cos(a + b*t)
-log(7*t + 5)*atan(3*t - 1)
a*log(d + e*t)*atan(a + c*t)
log(4 - 3*t)
log(a + b*t)
sin(9*t - 4)
sin(a + b*t)
-tan(9*t/2 - 0.5)
a*tan(b + 9*t/2)
tan(6*t - 6)/(6*t - 8)**0.333333333333333
-tan(a*t - a)/(-a*t - 8)**0.333333333333333
sin(2*sqrt(2)*sqrt(t))
sin(a**(3/2)*sqrt(t))
-tanh(6*t - 9)
a*tanh(b + c*t)
atan(t - 6)
atan(a + t)
log(3*t/5 - 1.2)
log(a + 3*t/5)
sin(3*t + 2)
sin(a + b*t)
tan(9*t + 1.5) - asinh(17*t/2 + 6)
b*asinh(a + 17*t/2) + tan(c + d*t)
log(t +

In [16]:
expr = sympify("-sin(2*t + 7)/(-4*t - 4)**0.333333333333333")
display(expr)
expanded_expr = sp.expand(expr)
term_dict = expanded_expr.as_coefficients_dict()
print(expanded_expr.func)
print(expanded_expr.args)

-sin(2*t + 7)/(-4*t - 4)**0.333333333333333

<class 'sympy.core.mul.Mul'>
(-1, (-4*t - 4)**(-0.333333333333333), sin(2*t + 7))


In [63]:
from sympy import symbols, sin, cos

# Define a sample expression
x, y = symbols('x y')
expr = sin(x*y) + cos(x**2) + 5

# Break down the expression into args and operators
def break_down_expr(expr):
    if expr.args:
        return expr.func, [break_down_expr(arg) for arg in expr.args]
    else:
        return expr

constant_idx = 0
    
def get_template(expr):
    global constant_idx
    constant_idx = 0
    broken_down_expr = break_down_expr(expr)
    #print("Broken Down Expression:", broken_down_expr)
    #
    # Reconstruct the original expression with constants replaced by symbols
    def reconstruct_expr(broken_down_expr, parent_func=None):
        global constant_idx
        if isinstance(broken_down_expr, tuple):
            func, args = broken_down_expr
#             if func == sp.Mul and args[0] == sympify('-1'):
#                 args[0] = 
#             else:
            return func(*[reconstruct_expr(arg, func) for arg in args])
        elif isinstance(broken_down_expr, (sp.Integer, sp.Float, sp.Rational)):
            if parent_func == sp.Mul and broken_down_expr == sympify('-1'):
                return broken_down_expr
            if parent_func == sp.Pow:
                return broken_down_expr
            symbol = symbols(f'a_{constant_idx}')
            constant_idx += 1
            return symbol
        else:
            return broken_down_expr
    #
    reconstructed_expr = reconstruct_expr(broken_down_expr)
    #print("Reconstructed Expression:", reconstructed_expr)
    return reconstructed_expr


display(get_template(expr))


fin = open("/home/mcwave/code/automath/calculus/datasets/function_evaluation.json")
lines = fin.readlines()
fin.close()

templates = {}

for i in range(10000): #len(lines)):
    data = json.loads(lines[i])
    try:
        s = data["f_t"][1]
        expr = sp.sympify(s)
        template = get_template(expr)
        print(expr)
        print(template)
    except:
        print("Cannot process", line)
    if i % 1000 == 0:
        print(i, "cases processed")

a_0 + sin(x*y) + cos(x**2)

-tan(7*t - 7)
-tan(a_0 + a_1*t)
0 cases processed
tan(6*t - 1)
tan(a_0 + a_1*t)
-sin(2*t + 7)/(-4*t - 4)**0.333333333333333
-sin(a_2 + a_3*t)/(a_0 + a_1*t)**0.333333333333333
-(6*t - 5)**0.333333333333333 - sin(6*t**5 - 9)
-(a_0 + a_1*t)**0.333333333333333 - sin(a_2 + a_3*t**5)
-tan(5*t - 5)
-tan(a_0 + a_1*t)
(-6*t**2 - 3)**(1/3)
(a_0 + a_1*t**2)**(1/3)
sin(16*t/3 + 6.66666666666667)
sin(a_0 + a_1*t)
(-6*t - 7)**(-4)
(a_0 + a_1*t)**(-4)
-cos(5*t - 6) - tanh(2*t + 7)
-cos(a_0 + a_1*t) - tanh(a_2 + a_3*t)
-tan(4*t - 5)
-tan(a_0 + a_1*t)
cos(6*t + 0.5)
cos(a_0 + a_1*t)
-log(7*t + 5)*atan(3*t - 1)
-log(a_2 + a_3*t)*atan(a_0 + a_1*t)
log(4 - 3*t)
log(a_0 + a_1*t)
sin(9*t - 4)
sin(a_0 + a_1*t)
-tan(9*t/2 - 0.5)
-tan(a_0 + a_1*t)
tan(6*t - 6)/(6*t - 8)**0.333333333333333
tan(a_2 + a_3*t)/(a_0 + a_1*t)**0.333333333333333
sin(2*sqrt(2)*sqrt(t))
sin(sqrt(2)*a_0*sqrt(t))
-tanh(6*t - 9)
-tanh(a_0 + a_1*t)
atan(t - 6)
atan(a_0 + t)
log(3*t/5 - 1.2)
log(a_0 + a_1*t)
sin(3*t + 2)
sin(a_0 + a_1*t)
tan

(t + 7)**0.333333333333333*cos(7*t)
(a_0 + t)**0.333333333333333*cos(a_1*t)
-sin(11*t/2 - 7)
-sin(a_0 + a_1*t)
sin(7*t + 9)*cos(9*t + 2)
sin(a_2 + a_3*t)*cos(a_0 + a_1*t)
-tan(5*t**5 - 8.66666666666667) - atan(t/3 + 6.66666666666667)
-tan(a_2 + a_3*t**5) - atan(a_0 + a_1*t)
1/((-t - 3)*log(-2*t))
1/((a_0 - t)*log(a_1*t))
-sin(7*t + 9)
-sin(a_0 + a_1*t)
sin(2*t**4 + 3)
sin(a_0 + a_1*t**4)
-tan(3*t/2 + 8)
-tan(a_0 + a_1*t)
-sin(3*t - 1)
-sin(a_0 + a_1*t)
cos(4*t - 1)
cos(a_0 + a_1*t)
log(log(-2*t**3 - 4)) - tan(6*t - 9)
log(log(a_2 + a_3*t**3)) - tan(a_0 + a_1*t)
sqrt(7*t - 8)
sqrt(a_0 + a_1*t)
acosh(6*t + 1)
acosh(a_0 + a_1*t)
exp(2 - 7*t)*log(t + 6)*cosh(1)
exp(a_1 + a_2*t)*log(a_3 + t)*cosh(a_0)
(8*t - 8)**(1/3)/(-t**5 - 8)**(1/3)
(a_1 + a_2*t)**(1/3)/(a_0 - t**5)**(1/3)
-tan(6*t**3 + 4)
-tan(a_0 + a_1*t**3)
-log(7*t + 2) + (5 - 7*t)**(-3)
-log(a_2 + a_3*t) + (a_0 + a_1*t)**(-3)
(-3*t - 5)**(1/3) + sin(3*t + 8)
(a_0 + a_1*t)**(1/3) + sin(a_2 + a_3*t)
log(-9*t/5 - 5.6)
log(a_0 + a_1*t)

-sin(5*t - 8)*csc(3*t**2 + 7)
-sin(a_2 + a_3*t)*csc(a_0 + a_1*t**2)
asin(2*t**4 - 1) - pi/4
pi*a_0 + asin(a_1 + a_2*t**4)
log(-3*t - 2)*cos(6*t - 7)
log(a_2 + a_3*t)*cos(a_0 + a_1*t)
exp(-3*t - 3)
exp(a_0 + a_1*t)
log(cos(21*t**2/5 + 2.4)) - atan(16*t/5 - 5.6)
log(cos(a_2 + a_3*t**2)) - atan(a_0 + a_1*t)
exp(3*t - 6)
exp(a_0 + a_1*t)
(-2*t - 3)**3 - 1/(t - 7)**3
(a_0 + a_1*t)**3 - 1/(a_2 + t)**3
(t + 9)**(-2)
(a_0 + t)**(-2)
-sin(6*t - 1)*sin(4*t**2 - 3)
-sin(a_0 + a_1*t)*sin(a_2 + a_3*t**2)
exp(-t/3 - 3)
exp(a_0 + a_1*t)
sqrt(15*t/2 + 7) + exp(3 - 2*t)
sqrt(a_0 + a_1*t) + exp(a_2 + a_3*t)
-sin(7*t - 4)
-sin(a_0 + a_1*t)
0.00136036803754789*exp(-32*t/5)
a_0*exp(a_1*t)
-cos(8*t + 7) - tan(5*t - 8)
-cos(a_0 + a_1*t) - tan(a_2 + a_3*t)
exp(-3*t)
exp(a_0*t)
cos((8*t + 8)**(1/3))
cos((a_0 + a_1*t)**(1/3))
1 - 7*t
a_0 + a_1*t
-cos(4*t + 1) + atan(3*t + 5)
-cos(a_0 + a_1*t) + atan(a_2 + a_3*t)
sin(exp(6*t + 3))
sin(exp(a_0 + a_1*t))
exp(-t - 5) - 1/(8 - 2*t)**2
exp(a_2 - t) - 1/(a_0 + a_1*t)*

1/((-2*t**2 - 5)**2*(-8*t**3 - 9)**0.333333333333333)
1/((a_0 + a_1*t**3)**0.333333333333333*(a_2 + a_3*t**2)**2)
exp(atan(9*t + 8)) + sin(5)
exp(atan(a_0 + a_1*t)) + sin(a_2)
(8*t + 8)**0.333333333333333*cos(7*t - 4)
(a_0 + a_1*t)**0.333333333333333*cos(a_2 + a_3*t)
-sin(7*t - 5)
-sin(a_0 + a_1*t)
log(0.5 - 17*t/2)
log(a_0 + a_1*t)
sinh(tan(5*t/2 + 6))
sinh(tan(a_0 + a_1*t))
-1.5874010519682*t**0.333333333333333*tan(7*t**3 - 3)
a_0*t**0.333333333333333*tan(a_1 + a_2*t**3)
exp(7*t + 8) + tan(5*t**5 - 7)
exp(a_0 + a_1*t) + tan(a_2 + a_3*t**5)
tan(t/2 - 6.5)
tan(a_0 + a_1*t)
sqrt(7*t + 3)*sec(2*t - 6)
sqrt(a_0 + a_1*t)*sec(a_2 + a_3*t)
-sin(7*t/2 - 3.5)
-sin(a_0 + a_1*t)
-atan(17*t**3/2 - 5.5)
-atan(a_0 + a_1*t**3)
(-8*t - 8)**(1/3)
(a_0 + a_1*t)**(1/3)
8*cot(6*t - 0.5)/(27*t**3)
a_0*cot(a_1 + a_2*t)/t**3
sin(6*t + 7)
sin(a_0 + a_1*t)
log(5*t + 2)
log(a_0 + a_1*t)
sin(15*t**2/2 + 6.5) + tan(6*t - 8)
sin(a_0 + a_1*t**2) + tan(a_2 + a_3*t)
tan(3*t)/atan(3*t**3 + 6)
tan(a_2*t)/atan(a_0 + a_

...

sin(exp(-9*t/2 - 7)) + 0.606530659712633*exp(-5*t)
a_0*exp(a_1*t) + sin(exp(a_2 + a_3*t))
(-3*t**4 - 1)**(-5)
(a_0 + a_1*t**4)**(-5)
cos(8*t - 1) + tan(7*t**4 - 6)
cos(a_0 + a_1*t) + tan(a_2 + a_3*t**4)
-sin(3*t + 5)
-sin(a_0 + a_1*t)
sin(0.301194211912202*exp(-38*t/5))
sin(a_0*exp(a_1*t))
cos(3*t + 3)
cos(a_0 + a_1*t)
(-8*t**2 - 1/2)**(-4)
(a_0 + a_1*t**2)**(-4)
cos(9*t/5 - 5) - atan(6*t + 7.8)
cos(a_2 + a_3*t) - atan(a_0 + a_1*t)
exp(t - 1)
exp(a_0 + t)
cos(7*t**3 + 8)*sech(4*t**4 + 4)
cos(a_0 + a_1*t**3)*sech(a_2 + a_3*t**4)
(4*t - 6)**(1/3)
(a_0 + a_1*t)**(1/3)
cos(tan(3*t**2/2 - 7.5))
cos(tan(a_0 + a_1*t**2))
tan(9*t + 5)
tan(a_0 + a_1*t)
tan(6*t + 5)
tan(a_0 + a_1*t)
exp(3*t - 3)
exp(a_0 + a_1*t)
-cosh(3*t/2 - 8.5) + 1/(16*(-t - 0.125)**2)
a_2/(a_3 - t)**2 - cosh(a_0 + a_1*t)
exp(6*t**2 + 5)
exp(a_0 + a_1*t**2)
-sin(5*t + 6)
-sin(a_0 + a_1*t)
log(6*t + 2)*cos(t - 4)
log(a_1 + a_2*t)*cos(a_0 + t)
(5 - 11*t/2)**2
(a_0 + a_1*t)**2
cos(4*t + 3)
cos(a_0 + a_1*t)
sin(7*t**3 + 3)
sin(a_

-cosh(4*t**4 - 3)*csch(2*t + 4)
-cosh(a_0 + a_1*t**4)*csch(a_2 + a_3*t)
exp(4*t + 2)
exp(a_0 + a_1*t)
log(t**2 + 5.5) - sin(9*t/2 - 8)
log(a_2 + t**2) - sin(a_0 + a_1*t)
1/(2401*t**20*log(3*t - 5))
a_0/(t**20*log(a_1 + a_2*t))
(4 - 2*t**4)*cos(4*t + 5)
(a_0 + a_1*t**4)*cos(a_2 + a_3*t)
1 - log(t - 2)
a_0 - log(a_1 + t)
atan(7*t**3 - 2.5)
atan(a_0 + a_1*t**3)
sqrt(-2*t**5 - 7)*cot(9*t + 7)
sqrt(a_0 + a_1*t**5)*cot(a_2 + a_3*t)
exp(-4*t)
exp(a_0*t)
tan(sqrt(4 - 4*t))
tan(sqrt(a_0 + a_1*t))
sin((8*t - 6)**0.166666666666667) + tan(t - 8)
sin((a_0 + a_1*t)**0.166666666666667) + tan(a_2 + t)
-asin(sin(6*t + 1))
-asin(sin(a_0 + a_1*t))
cos(17*t/2 - 5.5)
cos(a_0 + a_1*t)
(-4*t - 7)**0.333333333333333
(a_0 + a_1*t)**0.333333333333333
-tan(8*t - 4)
-tan(a_0 + a_1*t)
-atan(5*t**5)
-atan(a_0*t**5)
tan(4*t - 1)
tan(a_0 + a_1*t)
-tan(9*t + 1) - tan(6)
-tan(a_0) - tan(a_1 + a_2*t)
tan(3*t + 4)
tan(a_0 + a_1*t)
cos(7)*cos(5*t**5 - 2)
cos(a_0)*cos(a_1 + a_2*t**5)
exp(5 - 3*t**5)/(4*t**5 - 3)
exp(a_2 + 

(7*t**3 - 1)**0.166666666666667*cos(4*t - 6)
(a_0 + a_1*t**3)**0.166666666666667*cos(a_2 + a_3*t)
(7*t**2 + 5)**0.166666666666667 - tan(2*t + 6)
(a_0 + a_1*t**2)**0.166666666666667 - tan(a_2 + a_3*t)
cos(14*t**3/3 - 0.666666666666667)
cos(a_0 + a_1*t**3)
(7*t**2 - 2)**0.166666666666667*cos(3*t**4 + 3)
(a_0 + a_1*t**2)**0.166666666666667*cos(a_2 + a_3*t**4)
-atan(3*t + 3)
-atan(a_0 + a_1*t)
3125/(79235168*(-t - 0.894736842105263)**5)
a_0/(a_1 - t)**5
-(7*t - 8)**(1/3)*tan(4*t - 1)
-(a_0 + a_1*t)**(1/3)*tan(a_2 + a_3*t)
-(2*t + 4)**(1/3) - 1/(243*t**5)
a_2/t**5 - (a_0 + a_1*t)**(1/3)
(2*t**2 - 6)**(1/3) - tan(8*t + 6)
(a_0 + a_1*t**2)**(1/3) - tan(a_2 + a_3*t)
-tan(5*t + 3)
-tan(a_0 + a_1*t)
cos(11*t/2 + 8)
cos(a_0 + a_1*t)
-sqrt(4*t**2 - 5) + sin(8*t**4 - 3)
-sqrt(a_0 + a_1*t**2) + sin(a_2 + a_3*t**4)
(-7*t - 2)**(1/3)*cos(7*t + 8)
(a_0 + a_1*t)**(1/3)*cos(a_2 + a_3*t)
tan(9*t - 5)*tan(2*t**5 - 4)
tan(a_0 + a_1*t)*tan(a_2 + a_3*t**5)
cos(4*t - 7)
cos(a_0 + a_1*t)
(8 - 2*t**5)**4*(-5*t -

2.54950975679639*sqrt(0.307692307692308*t**5 + 1)*log(8.5 - 17*t/2)
a_0*sqrt(a_1 + a_2*t**5)*log(a_3 + a_4*t)
cos(8*t + 2)
cos(a_0 + a_1*t)
sin(2*t**3 + 4)/(4*t - 3)**0.333333333333333
sin(a_2 + a_3*t**3)/(a_0 + a_1*t)**0.333333333333333
sqrt(5*t - 8)*exp(-2*t**5 - 5)
sqrt(a_0 + a_1*t)*exp(a_2 + a_3*t**5)
7 - 5*t
a_0 + a_1*t
exp(8*t + 1)*cos(8*t**3 - 1)
exp(a_2 + a_3*t)*cos(a_0 + a_1*t**3)
cos(6*t - 8.5)
cos(a_0 + a_1*t)
-tan(7*t/2 + 2.5)
-tan(a_0 + a_1*t)
-sin(9*t**3 - 2)
-sin(a_0 + a_1*t**3)
log(exp(8 - 5*t)) + tan(5*t**2 + 5)
log(exp(a_0 + a_1*t)) + tan(a_2 + a_3*t**2)
81*t**4/16
a_0*t**4
exp(3*t + 9)*log(2 - 8*t)
exp(a_0 + a_1*t)*log(a_2 + a_3*t)
-tan(2*t**2 - 1)
-tan(a_0 + a_1*t**2)
-exp(-t**2 - 6)/atan(t - 4)
-exp(a_1 - t**2)/atan(a_0 + t)
tan(cos(15*t**2/2 + 0.5))
tan(cos(a_0 + a_1*t**2))
0.000500451433440611*exp(32*t/5) + sin(7*t**3/5 + 0.8)
a_0*exp(a_1*t) + sin(a_2 + a_3*t**3)
2040.63314888554*sqrt(3)*sqrt(t + 1.0)*exp(-26*t/3)
sqrt(3)*a_0*sqrt(a_1 + t)*exp(a_2*t)
tanh(9*t**2 

sqrt(8*t**2 - 6)
sqrt(a_0 + a_1*t**2)
-atan(24*t/5 - 4.8) + (-5*t - 1)**(-2)
-atan(a_2 + a_3*t) + (a_0 + a_1*t)**(-2)
(-5*t - 4)**(1/3)
(a_0 + a_1*t)**(1/3)
tan(sin(t + 4))
tan(sin(a_0 + t))
tan(sqrt(8 - 3*t)) - sin(8)
-sin(a_0) + tan(sqrt(a_1 + a_2*t))
(8 - 3*t**2)**2*(8*t + 7)**0.333333333333333
(a_0 + a_1*t)**0.333333333333333*(a_2 + a_3*t**2)**2
-sin(4*t**5) - cos(3*t - 9)
-sin(a_2*t**5) - cos(a_0 + a_1*t)
cos(t/2 + 3.5)*tan(6*t + 1)
cos(a_0 + a_1*t)*tan(a_2 + a_3*t)
-sin(8*t**2/5 + 7.4)
-sin(a_0 + a_1*t**2)
cos(8*t - 1.5) - tan(7*t/2 - 7)
cos(a_2 + a_3*t) - tan(a_0 + a_1*t)
exp(-8)/log(3*t - 6)
exp(a_2)/log(a_0 + a_1*t)
665.141633044362*exp(t/2)
a_0*exp(a_1*t)
-tan(7*t**4 - 4)
-tan(a_0 + a_1*t**4)
(t/2 + 1)**(-4)
(a_0 + a_1*t)**(-4)
sqrt(3*t - 6)/(-4*t - 5)**3
sqrt(a_0 + a_1*t)/(a_2 + a_3*t)**3
tan(4*t - 5)
tan(a_0 + a_1*t)
log(11*t/2 - 9)
log(a_0 + a_1*t)
(3*t + 4)**(1/9)
(a_0 + a_1*t)**(1/9)
-7**(1/3)*t**(1/3)
-7**(1/3)*t**(1/3)
-cot(2*t + 3)*sinh(t - 7)
-cot(a_0 + a_1*t)*sinh(a

In [60]:
break_down_expr(sympify("(-6*t**2 - 3)**(1/3)"))

(sympy.core.power.Pow,
 [(sympy.core.add.Add,
   [-3, (sympy.core.mul.Mul, [-6, (sympy.core.power.Pow, [t, 2])])]),
  1/3])

In [ ]:
import time
import torch
import random
seq = [0] * (257 * 768)
start_time = time.time()
for i in range(10):
    j = 0
    while j < len(seq):
        j += 1
print(time.time()-start_time)